In [1]:
import os
import torch
import random
import shutil
import time
import numpy as np
from tqdm import tqdm 
from matplotlib import pyplot as plt
from typing import Optional, List, Union, Dict

import ase
from ase.io import read, write
from ase import Atoms
from ase.optimize.lbfgs import LBFGS
from ase.optimize import QuasiNewton

import schnetpack as spk
from schnetpack import properties
from schnetpack.interfaces.ase_interface import AtomsConverter, AseInterface
from schnetpack.interfaces.ensemble_calculator import EnsembleCalculator, SpkCalculator
from schnetpack.interfaces.batchwise_optimization import ASEBatchwiseLBFGS, BatchwiseCalculator

In [2]:
# TODO: Markdown comments, uncertainty evaluation example

In [3]:
model_path_0 = "/home/jonas/Documents/schnetpack/tests/testdata/md_ethanol.model"
model_path_1 = "/home/jonas/Documents/schnetpack/tests/testdata/md_ethanol.model"

# set device
device = torch.device("cuda")

# define neighbor list
cutoff = 5.0
nbh_list=spk.transform.MatScipyNeighborList(cutoff=cutoff)

# Set up calculator
calculator = EnsembleCalculator(
    model=[model_path_0, model_path_1],
    neighbor_list=nbh_list,
    energy_unit="kcal/mol",
    position_unit="Angstrom",
    device=device,
)

In [4]:
input_structure_file = "../../tests/testdata/md_ethanol.xyz"
random.seed(42)

# Setup directory
if not os.path.exists('howto_ensemble_relaxations_outputs'):
    os.makedirs('howto_ensemble_relaxations_outputs')
working_dir = "howto_ensemble_relaxations_outputs/relax"
if os.path.exists(working_dir):
    shutil.rmtree(working_dir)
os.makedirs(working_dir)

# load initial structure
mol = read(input_structure_file)
pos = mol.get_positions()
# distort the structure
for n in range(pos.shape[0]):
    pos[n] = pos[n] * random.uniform(0.95,1.05)
molecule = Atoms(positions=pos, numbers=mol.get_atomic_numbers())
molecule.calc = calculator

In [5]:
# run relaxation
optimize_file = os.path.join(working_dir, "optimization")
optimizer = LBFGS(
    molecule,
    trajectory="{:s}.traj".format(optimize_file),
    restart="{:s}.pkl".format(optimize_file),
)
optimizer.run(fmax=0.001, steps=1000)

       Step     Time          Energy         fmax
LBFGS:    0 14:58:07    -4208.208776        9.6160
LBFGS:    1 14:58:07    -4209.343051        5.9222
LBFGS:    2 14:58:07    -4209.930915        2.7088
LBFGS:    3 14:58:07    -4210.075855        1.5142
LBFGS:    4 14:58:07    -4210.117326        0.5858
LBFGS:    5 14:58:07    -4210.156747        0.4284
LBFGS:    6 14:58:07    -4210.189389        0.4514
LBFGS:    7 14:58:07    -4210.199779        0.3549
LBFGS:    8 14:58:07    -4210.206293        0.2580
LBFGS:    9 14:58:08    -4210.209682        0.1759
LBFGS:   10 14:58:08    -4210.212134        0.1565
LBFGS:   11 14:58:08    -4210.213689        0.1077
LBFGS:   12 14:58:08    -4210.214780        0.0776
LBFGS:   13 14:58:08    -4210.215383        0.0685
LBFGS:   14 14:58:08    -4210.215853        0.0723
LBFGS:   15 14:58:08    -4210.216322        0.0650
LBFGS:   16 14:58:08    -4210.216701        0.0550
LBFGS:   17 14:58:08    -4210.216910        0.0435
LBFGS:   18 14:58:08    -4210.21

True